# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,avarua,-21.2078,-159.7750,26.03,94,20,3.09,CK,1683513351
1,1,blackmans bay,-43.0167,147.3167,13.07,70,97,3.91,AU,1683513351
2,2,ribeira grande,38.5167,-28.7000,17.65,98,100,3.58,PT,1683513352
3,3,adamstown,-25.0660,-130.1015,21.67,59,100,10.11,PN,1683513352
4,4,margaret river,-33.9500,115.0667,20.19,43,100,7.66,AU,1683513352


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot = city_data_df.hvplot.points('Lng','Lat',geo=True, tiles = 'OSM', s='Humidity', color='City')

# Display the map
# YOUR CODE HERE
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_city_data_df = city_data_df[
    (city_data_df['Max Temp']>= 20) &
    (city_data_df['Max Temp']< 25) &
    (city_data_df['Cloudiness']== 0) &
    (city_data_df['Wind Speed']< 4.5)
]

# Drop any rows with null values
# YOUR CODE HERE - there are no null values in the results
# Display sample data
# YOUR CODE HERE
ideal_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
149,149,wurtulla,-26.7568,153.1256,20.72,39,0,3.58,AU,1683513416
215,215,nahiyat ash shinafiyah,31.5874,44.6602,24.28,25,0,3.82,IQ,1683513445
240,240,klyuchi,52.2667,79.1667,22.57,26,0,1.23,RU,1683513455
257,257,nova xavantina,-14.6667,-52.3500,21.92,73,0,0.99,BR,1683513462
268,268,tolanaro,-25.0319,46.9987,22.04,88,0,3.09,MG,1683513466
281,281,toliara,-23.3500,43.6667,21.89,69,0,2.05,MG,1683513385
296,296,gebeit,21.0667,36.3167,21.95,36,0,2.30,SD,1683513482
301,301,pine bluff,34.2284,-92.0032,22.40,90,0,1.54,US,1683513484
390,390,abadan,30.3392,48.3043,23.97,64,0,2.06,IR,1683513529
447,447,shorkot,31.9099,70.8774,24.09,38,0,3.36,PK,1683513552


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_city_data_df[['City', 'Country','Lat','Lng', 'Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = " "
# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
149,wurtulla,AU,-26.7568,153.1256,39,
215,nahiyat ash shinafiyah,IQ,31.5874,44.6602,25,
240,klyuchi,RU,52.2667,79.1667,26,
257,nova xavantina,BR,-14.6667,-52.3500,73,
268,tolanaro,MG,-25.0319,46.9987,88,
281,toliara,MG,-23.3500,43.6667,69,
296,gebeit,SD,21.0667,36.3167,36,
301,pine bluff,US,34.2284,-92.0032,90,
390,abadan,IR,30.3392,48.3043,64,
447,shorkot,PK,31.9099,70.8774,38,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000 # YOUR CODE HERE
categories = 'accommodation.hotel'

params = {
    # YOUR CODE HERE
    'apiKey':geoapify_key,
    'format':'json',
    'categories': categories

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    # Lat = hotel_df.iloc[row]['Lat']
    # Lng = hotel_df.iloc[row]['Lng']
    Lat = row['Lat']
    Lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}" # YOUR CODE HERE
    params["bias"] = f"proximity:{Lng},{Lat}" # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params).json() # YOUR CODE HERE
    
    # Convert the API response to JSON format
    # name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
wurtulla - nearest hotel: Currimundi Hotel
nahiyat ash shinafiyah - nearest hotel: No hotel found
klyuchi - nearest hotel: No hotel found
nova xavantina - nearest hotel: Hotel Morada do Sol
tolanaro - nearest hotel: Hôtel Mahavokey
toliara - nearest hotel: Ambary
gebeit - nearest hotel: No hotel found
pine bluff - nearest hotel: Days Inn & Suites by Wyndham Pine Bluff
abadan - nearest hotel: خانه معلم
shorkot - nearest hotel: Paradise Hotel
khairpur - nearest hotel: No hotel found
rawah - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
149,wurtulla,AU,-26.7568,153.1256,39,Currimundi Hotel
215,nahiyat ash shinafiyah,IQ,31.5874,44.6602,25,No hotel found
240,klyuchi,RU,52.2667,79.1667,26,No hotel found
257,nova xavantina,BR,-14.6667,-52.3500,73,Hotel Morada do Sol
268,tolanaro,MG,-25.0319,46.9987,88,Hôtel Mahavokey
281,toliara,MG,-23.3500,43.6667,69,Ambary
296,gebeit,SD,21.0667,36.3167,36,No hotel found
301,pine bluff,US,34.2284,-92.0032,90,Days Inn & Suites by Wyndham Pine Bluff
390,abadan,IR,30.3392,48.3043,64,خانه معلم
447,shorkot,PK,31.9099,70.8774,38,Paradise Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot2 = hotel_df.hvplot.points('Lng','Lat',geo=True, tiles = 'OSM', s='Humidity', color='City', hover_cols=['Country','Hotel Name'])

# Display the map
# YOUR CODE HERE
map_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)